In [1]:
# https://nlp.johnsnowlabs.com/docs/en/annotators#languagedetectordl

# maven - com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.3
# pypi - spark-nlp, vaderSentiment, boto3==1.9.157

# example on sentiment analysis
  # https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/SentimentDL_train_multiclass_sentiment_classifier.ipynb
# https://nlp.johnsnowlabs.com/docs/en/pipelines#multi-language

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkFiles
from pyspark.ml.feature import (
    StringIndexer,
    HashingTF, 
    IDF
)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes

In [3]:
# get or create Spark session

app_name = "news_analysis"

spark = SparkSession.builder.appName(app_name) \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.3") \
    .getOrCreate()

In [4]:
file_uri = "/mnt/mnt_s3/BusinessTrainData.csv"

trainDataset = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(file_uri)
display(trainDataset)

_c0,index,category,headline,authors,link,short_description,date,short_description_new,label
148341,170325,BUSINESS,"Stores Open On Christmas Eve Include Walmart, Best Buy, Target, Among Others",Caroline Fairchild,https://www.huffingtonpost.com/entry/stores-open-on-christmas-eve-2012_us_5bb2e167e4b0480ca65de749,leave christmas shopping minute worry americas retailers got covered,2012-12-22,leave christmas shopping minute worry americas retailers got covered,Negative
94803,106274,BUSINESS,Trusting the Crowd and the Machines,"Despina Katsikakis, ContributorGlobally renowned expert on how to transform the workplace to ...",https://www.huffingtonpost.com/entry/trusting-the-crowd-and-th_b_6166406.html,business environment future needs trust people technology provide flexibility choice employees connect complementary skills network work challenges learn fast unlock passion improve performance,2014-11-19,business environment future needs trust people technology provide flexibility choice employees connect complementary skills network work challenges learn fast unlock passion improve performance,Positive
86152,90632,BUSINESS,"""The Sudden Implosion of """"Secret"""" May Reveal A Secret: Is Silicon Valley Out Of Touch Or Just Losing It's Touch?""","Tony Potts, ContributorPartner-SierraMaya360/PartnersTrust/Prev-Access Hollywood",https://www.huffingtonpost.com/entry/the-sudden-implosion-of-s_b_7310388.html,look secret timeline demise started august inverted hockey stick trajectory came days raised july investors nightmare doubt,2015-05-19,look secret timeline demise started august inverted hockey stick trajectory came days raised july investors nightmare doubt,Negative
40379,41278,BUSINESS,He's Grounded! Delta Bans Obnoxious Trump Supporter For Life,Carla Herreria,https://www.huffingtonpost.com/entry/delta-bans-trump-supporter_us_583ce595e4b0860d611640a2,ceo says passengers flight given refunds,2016-11-29,ceo says passengers flight given refunds,Neutral
34901,35480,BUSINESS,DON'T PANIC! That Bacon Shortage You Keep Hearing About Isn't Real,Ed Mazza,https://www.huffingtonpost.com/entry/no-bacon-shortage_us_5892dfbae4b0bf5206e63393,america isnt running bacon,2017-02-02,america running bacon,Neutral
57004,58878,BUSINESS,One Map That Shows How The Middle Class Is Getting Squeezed,Shane Ferro,https://www.huffingtonpost.com/entry/us-middle-class-squeezed_us_5734ea76e4b060aa7819c3c6,people arent earning,2016-05-13,people arent earning,Neutral
177642,200136,BUSINESS,Facebook Is No Longer Cool,"Max Kolonko, Contributor",null,null,null,null,null
New York television producer,director,"author and the U.S. te...""",https://www.huffingtonpost.com/entry/facebook-public_us_5bb2b460e4b0480ca659ddfa,facebook billionaire mark zuckerberg joined establishment corporate giant cheated luring concept global friendship turned lined corporate greed,2012-02-05,facebook billionaire mark zuckerberg joined establishment corporate giant cheated luring concept global friendship turned lined corporate greed,Negative,null,null
158327,180430,MONEY,Master the Five Factors That Feed FICO This Fall,"Jeanne Kelly, Contributor",null,null,null,null,null
Credit & Identity Theft Expert,Author,"CEO of The Kelly Gro...""",https://www.huffingtonpost.com/entry/master-the-five-factors-t_us_5b9c372ce4b03a1dcc7d278f,credit score taken summer vacation time commitments changes,2012-09-04,credit score summer vacation time commitments changes,Positive,null,null


In [5]:
file_uri = "/mnt/mnt_s3/BusinessTestData.csv"

testDataset = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(file_uri)
display(testDataset)

_c0,index,category,headline,authors,link,short_description,date,short_description_new,label
106103,127278,MONEY,Future Shock: Personal Finance Technology,"Jim Gibson, Contributor",null,null,null,null,null
Personal Finance writer,"Your Finances Simplified""",https://www.huffingtonpost.com/entry/future-shockpersonal-fina_us_5b9df118e4b03a1dcc8f2e62,land way banking receipts growing trend alternative payment,2014-03-24,land banking receipts growing trend alternative payment,Positive,null,null,null
84389,87560,BUSINESS,City Of Charleston Steps Up To Help Victims' Families In A Big Way,Andrew Lord,https://www.huffingtonpost.com/entry/business-charleston-victims_n_7644912.html,donations pouring local businesses,2015-06-23,donations pouring local businesses,Neutral
116537,137877,BUSINESS,The Shoestring Guide to Brand Publishing,null,https://www.huffingtonpost.com/entry/brand-publishing-budget_us_5bb302d8e4b0480ca66147cf,contently creating stream content stands crowd brand map brand key,2013-11-30,contently creating stream content stands crowd brand map brand key,Positive
60976,63051,BUSINESS,One Of Ben Carson's Craziest Ideas Is Coming True,Ben Walsh,https://www.huffingtonpost.com/entry/us-tax-haven_us_56f56a7be4b014d3fe22f892,worlds hottest new tax haven,2016-03-26,worlds hottest new tax haven,Neutral
174267,196702,MONEY,St. Patrick's Day 2012: Saturday Holiday A Boon For Bars And Retailers,null,https://www.huffingtonpost.com/entry/st-patricks-day-spending_us_5b9b7184e4b03a1dcc779415,celebrating day likely entail spending money st patricks day far expensive holiday,2012-03-13,celebrating day likely entail spending money patricks day far expensive holiday,Positive
92267,101497,BUSINESS,'Breastaurants' Thrive As The Restaurant Industry Struggles,Jillian Berman,https://www.huffingtonpost.com/entry/breastaurants-growth_n_6443274.html,serve burger cleavage crowds come pouring,2015-01-13,serve burger cleavage crowds come pouring,Neutral
97355,111221,BUSINESS,What Should I Do if an Employee Is a Liar?,"Russ Warner, ContributorVP Marketing at Converus, makers of EyeDetect, innovative solu...",https://www.huffingtonpost.com/entry/what-should-i-do-if-an-em_b_5864410.html,need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow know employee going repeat past behavior illicit drug use theft bribery,2014-09-23,need careful assume individuals guaranteed repeat past behavior assumptions limit persons ability learn grow employee going repeat past behavior illicit drug use theft bribery,Positive
54038,55728,BUSINESS,IEX Wins SEC Approval As U.S. Stock Exchange,null,https://www.huffingtonpost.com/entry/iex-sec-stock_us_576499dee4b015db1bc98569,builtin delay protect investors traders use tactics depicted michael lewis book flash boys wall street revolt,2016-06-18,builtin delay protect investors traders use tactics depicted michael lewis book flash boys wall street revolt,Positive
135872,157630,BUSINESS,"Mohammed Sohel Rana, Bangladesh Factory Owner, Faces Murder Complaint As Death Toll Rises To 622","Reuters, Reuters",https://www.huffingtonpost.com/entry/mohammed-sohel-rana-murder_us_5bb2f3b6e4b0480ca65f6045,guilty killings highest punishment capital punishment said abdul huq,2013-05-05,guilty killings highest punishment capital punishment said abdul huq,Negative


In [6]:
testDataset.toPandas()

,_c0,index,category,headline,authors,link,short_description,date,short_description_new,label
0,106103,127278,MONEY,Future Shock: Personal Finance Technology,"Jim Gibson, Contributor\n",None,None,None,None,None
1,Personal Finance writer,"Your Finances Simplified""",https://www.huffingtonpost.com/entry/future-sh...,land way banking receipts growing trend altern...,2014-03-24,land banking receipts growing trend alternati...,Positive,None,None,None
2,84389,87560,BUSINESS,City Of Charleston Steps Up To Help Victims' F...,Andrew Lord,https://www.huffingtonpost.com/entry/business-...,donations pouring local businesses,2015-06-23,donations pouring local businesses,Neutral
3,116537,137877,BUSINESS,The Shoestring Guide to Brand Publishing,None,https://www.huffingtonpost.com/entry/brand-pub...,contently creating stream content stands crowd...,2013-11-30,contently creating stream content stands crowd...,Positive
4,60976,63051,BUSINESS,One Of Ben Carson's Craziest Ideas Is Coming True,Ben Walsh,https://www.huffingtonpost.com/entry/us-tax-ha...,worlds hottest new tax haven,2016-03-26,worlds hottest new tax haven,Neutral
5,174267,196702,MONEY,St. Patrick's Day 2012: Saturday Holiday A Boo...,None,https://www.huffingtonpost.com/entry/st-patric...,celebrating day likely entail spending money s...,2012-03-13,celebrating day likely entail spending money ...,Positive
6,92267,101497,BUSINESS,'Breastaurants' Thrive As The Restaurant Indus...,Jillian Berman,https://www.huffingtonpost.com/entry/breastaur...,serve burger cleavage crowds come pouring,2015-01-13,serve burger cleavage crowds come pouring,Neutral
7,97355,111221,BUSINESS,What Should I Do if an Employee Is a Liar?,"Russ Warner, ContributorVP Marketing at Conver...",https://www.huffingtonpost.com/entry/what-shou...,need careful assume individuals guaranteed rep...,2014-09-23,need careful assume individuals guaranteed rep...,Positive
8,54038,55728,BUSINESS,IEX Wins SEC Approval As U.S. Stock Exchange,None,https://www.huffingtonpost.com/entry/iex-sec-s...,builtin delay protect investors traders use ta...,2016-06-18,builtin delay protect investors traders use ta...,Positive
9,135872,157630,BUSINESS,"Mohammed Sohel Rana, Bangladesh Factory Owner,...","Reuters, Reuters",https://www.huffingtonpost.com/entry/mohammed-...,guilty killings highest punishment capital pun...,2013-05-05,guilty killings highest punishment capital pun...,Negative


In [7]:
trainDataset.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5666 entries, 0 to 5665
Data columns (total 10 columns):
_c0 5666 non-null object
index 5666 non-null object
category 5666 non-null object
headline 5666 non-null object
authors 4986 non-null object
link 4737 non-null object
short_description 4297 non-null object
date 3930 non-null object
short_description_new 3830 non-null object
label 3807 non-null object
dtypes: object(10)
memory usage: 442.7+ KB

In [8]:
trainDataset = trainDataset.na.drop()

In [9]:
trainDataset= trainDataset.filter((trainDataset["label"]=="Positive") | (trainDataset["label"]=="Negative") | (trainDataset["label"]=="Neutral"))


In [10]:
trainDataset.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 10 columns):
_c0 3118 non-null object
index 3118 non-null object
category 3118 non-null object
headline 3118 non-null object
authors 3118 non-null object
link 3118 non-null object
short_description 3118 non-null object
date 3118 non-null object
short_description_new 3118 non-null object
label 3118 non-null object
dtypes: object(10)
memory usage: 243.7+ KB

In [11]:
trainDataset.toPandas()['label'].value_counts()

Out[11]: Positive 1483
Negative 843
Neutral 792
Name: label, dtype: int64

In [12]:
trainDataset.show()

+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
 _c0| index|category| headline| authors| link| short_description| date|short_description_new| label|
+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
148341|170325|BUSINESS|Stores Open On Ch...| Caroline Fairchild|https://www.huffi...|leave christmas s...|2012-12-22| leave christmas s...|Negative|
 94803|106274|BUSINESS|Trusting the Crow...|Despina Katsikaki...|https://www.huffi...|business environm...|2014-11-19| business environm...|Positive|
 86152| 90632|BUSINESS|"The Sudden Implo...|Tony Potts, Contr...|https://www.huffi...|look secret timel...|2015-05-19| look secret timel...|Negative|
 40379| 41278|BUSINESS|He's Grounded! De...| Carla Herreria|https://www.huffi...|ceo says passenge...|2016-11-29| ceo says passenge...| Neutral|
 34901| 35480|BUSINESS|DON'T PANIC! That...| Ed Mazza|https://www.huffi...|america isnt runn...|2017-02-02| america running ...| Neutral|
 57004| 58878|BUSINESS|One Map That Show...| Shane Ferro|https://www.huffi...|people arent earning|2016-05-13| people arent earning| Neutral|
 88041| 93760|BUSINESS|The High Cost of ...|Leo W. Gerard, Co...|https://www.huffi...|raises course cos...|2015-04-13| raises course cos...|Negative|
135953|157711|BUSINESS|Bangladesh Asks E...| Reuters, Reuters|https://www.huffi...|adds ruma paul se...|2013-05-04| adds ruma paul se...| Neutral|
150904|172914|BUSINESS|Anti-Austerity Ma...| Reuters, Reuters|https://www.huffi...|students occupied...|2012-11-24| students occupied...|Negative|
 64654| 66880|BUSINESS|Martin Shkreli Wa...| Sebastian Murdock|https://www.huffi...|horrible person h...|2016-02-11| horrible person h...|Negative|
129466|151110|BUSINESS|Expected Decline ...| Reuters, Reuters|https://www.huffi...|july reuters unex...|2013-07-13| july reuters unex...|Positive|
102534|122343|BUSINESS|The Potential Pro...|Dan Solin, Contri...|https://www.huffi...|case poses dilemm...|2014-05-20| case poses dilemm...|Positive|
 92274|101505|BUSINESS|4 Ways the Free M...|Patrick Donohoe, ...|https://www.huffi...|human genius come...|2015-01-13| human genius come...|Positive|
100327|117464|BUSINESS|Leader-as-Perform...|Dr. Jim Taylor, C...|https://www.huffi...|performance vital...|2014-07-14| performance vital...|Positive|
164053|186293| MONEY|Consumer Debt Del...| Reuters, Reuters|https://www.huffi...|washington july r...|2012-07-03| washington july r...|Positive|
 83958| 86856|BUSINESS|You Don't Need Ro...|"Valeh Nazemoff, ...|https://www.huffi...|look people like ...|2015-07-01| look people like ...|Positive|
 57872| 59788|BUSINESS|Airline Profits S...| Chris D'Angelo|https://www.huffi...| holy baggage fees|2016-05-03| holy baggage fees| Neutral|
 77222| 79814|BUSINESS|How To Deal With ...|Molly Owens, Cont...|https://www.huffi...|biggest challenge...|2015-09-19| biggest challenge...|Positive|
 65212| 67453|BUSINESS|Obama Ridiculed F...| Shahien Nasiripour|https://www.huffi...|education departm...|2016-02-05| education departm...|Negative|
126245|147817|BUSINESS|States Profiting ...|24/7 Wall St., 24...|https://www.huffi...|like tax foundati...|2013-08-17| like tax foundati...|Positive|
+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
only showing top 20 rows

In [13]:
testDataset = testDataset.na.drop()

In [14]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline

sparknlp.start()

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [15]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("short_description_new")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ]

In [16]:
trainDataset

In [17]:
trainDataset.filter("short_description_new IS NULL").count()

In [18]:

# pipelineModel = pipeline.fit(trainDataset.filter("label IN ('Positive','Negative')"))


In [19]:
pipelineModel.save("/mnt/mnt_s3/sentimentdl_pipeline_2")

In [20]:
loadedPipeline = PipelineModel.load("/mnt/mnt_s3/sentimentdl_pipeline_2")

In [21]:
import pandas as pd
pdf = pd.DataFrame({"text": [
  "This is so great! How are you", 
  "No way, never again!", 
  "Yes, bring on more!"
]})
df = spark.createDataFrame(pdf)

results = pipelineModel.transform(df)
display(
  results
)

In [22]:
display(
  results.selectExpr("text", "class.result")
)

In [23]:
analyzer.polarity_scores("This is so great! How are you")

In [24]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

@F.udf(returnType="struct<neg:double,neu:double,pos:double,compound:double>")
def getVADER(text):
	return analyzer.polarity_scores(text)

new_results = results.withColumn("vaderSentiment", getVADER("text"))
display(new_results)